In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os

sys.path.insert(0, "../apricot/model")

In [3]:
import torch
import torch.nn as nn

from model import CifarClassifier
import model
import pytorch2keras

from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np

/home/jjsohn/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jjsohn/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jjsohn/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jjsohn/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: 

In [2]:
img_size = 32
idlm_path = './models/CIFAR10_CifarClassifier.pth'

In [3]:
org_net = CifarClassifier(img_size = 3*32*32)
org_net.load_state_dict(torch.load(idlm_path))
org_net.eval()
org_net_org_weights = org_net.dense_layers[-1][-1].weight.clone()

CifarClassifier(
  (conv_layers): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (dense_layers): Sequential(
    (0): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=1024, out_features=512, bias=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=512, out_features=10, bias=True)
    )
  )
)

In [11]:
for layer in org_net.conv_layers[0]:
    if type(layer) == nn.Conv2d:
        print(layer.weight)

Parameter containing:
tensor([[[[ 0.0303,  0.2385,  0.2139],
          [ 0.0067, -0.1750,  0.0638],
          [ 0.1029,  0.0129, -0.0354]],

         [[-0.1706, -0.1518, -0.1546],
          [-0.0834, -0.1887,  0.0242],
          [-0.0789, -0.0695, -0.1823]],

         [[-0.1093,  0.0612,  0.0718],
          [-0.0521,  0.0259,  0.1292],
          [ 0.0935, -0.0489, -0.0781]]],


        [[[ 0.0280,  0.1058,  0.1323],
          [-0.0297, -0.0357, -0.3150],
          [-0.0578, -0.0934,  0.0427]],

         [[ 0.1431,  0.2659,  0.2494],
          [-0.1142, -0.1243, -0.1658],
          [-0.1367,  0.0363, -0.0245]],

         [[ 0.2578, -0.1678,  0.0977],
          [-0.0521, -0.0551, -0.0103],
          [-0.0262,  0.0837, -0.1140]]],


        [[[-0.0570, -0.2920,  0.0649],
          [-0.2321, -0.0058, -0.0722],
          [ 0.1853, -0.1142, -0.1197]],

         [[-0.0230,  0.0724,  0.0617],
          [ 0.0149, -0.0870, -0.1538],
          [-0.1856,  0.0544, -0.0559]],

         [[ 0.1704,  0

In [13]:
# parameters
seed_val = 0
proportion = 0.05

np.random.seed(seed_val)

In [15]:
def inject_faults(layer):
    if type(layer) in (nn.Linear, nn.Conv2d):
        org_weights_np = layer.weight.detach().numpy()
        ow_stdev = np.std(org_weights_np)
        random_faults = np.random.binomial(1, proportion, size=org_weights_np.shape)
        new_weights_np = (1-random_faults)*org_weights_np + random_faults*ow_stdev*np.random.randn(*org_weights_np.shape)
        layer.weight = nn.Parameter(torch.from_numpy(new_weights_np).float())

In [11]:
def inject_faults_for_keras_mdl(layer):
    import re
    targeting_clname_pattns = ['Dense*', 'Conv*'] #if not target_all else None
    is_target = lambda clname,targets: (targets is None) or any([bool(re.match(t,clname)) for t in targets])
    
    class_name = type(layer).__name__
    if is_target(class_name, targeting_clname_pattns):
        #org_weights_np = layer.weight.detach().numpy()
        org_weights_np, org_bias = layer.get_weights()
        
        ow_stdev = np.std(org_weights_np)
        random_faults = np.random.binomial(1, proportion, size = org_weights_np.shape)
        new_weights_np = (1-random_faults)*org_weights_np + random_faults*ow_stdev*np.random.randn(*org_weights_np.shape)
        #layer.weight = nn.Parameter(torch.from_numpy(new_weights_np).float())
        layer.set_weights([new_weights_np, org_bias])

In [16]:
for seed_val in range(30):
    # plant seed
    np.random.seed(seed_val)
    
    # reset network
    org_net.load_state_dict(torch.load(idlm_path))
    
    # inject faults (over all layers)
    org_net.apply(inject_faults)

    # save network
    input_var = torch.randn(1, 3, 32, 32)
    k_model = pytorch2keras.pytorch_to_keras(org_net, input_var, [(3, 32, 32)], verbose=True)
    k_model.save('./faulty_models/cifar10_allLayers_seed%d.h5' % seed_val)

INFO:pytorch2keras:Converter is called.
DEBUG:pytorch2keras:Input_names:
DEBUG:pytorch2keras:['input_0']
DEBUG:pytorch2keras:Output_names:
DEBUG:pytorch2keras:['output_0']
INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:[(3, 32, 32)]
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input_0.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> output_0.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight conv_layers.0.0.bias with shape (16,).
DEBUG:onnx2keras:Found weight conv_layers.0.0.weight with shape (16, 3, 3, 3).
DEBUG:onnx2keras:Found weight conv_layers.0.1.bias with shape (16,).
DEBUG:onnx2keras:Found weight conv_layers.0.1.num_batches_tracked with shape ().
DEBUG:onnx2keras:Found weight conv_layers.0.1.running_mean with shape (16,).
DEBUG:onnx2keras:Found weight conv_layers.0.1.running_var with shape (16,).
DEBUG:onnx2keras:Found weight conv_layers.0.1.weight with shape (16,).
DEBUG:onnx2kera

graph(%input_0 : Float(1, 3, 32, 32),
      %conv_layers.0.0.weight : Float(16, 3, 3, 3),
      %conv_layers.0.0.bias : Float(16),
      %conv_layers.0.1.weight : Float(16),
      %conv_layers.0.1.bias : Float(16),
      %conv_layers.0.1.running_mean : Float(16),
      %conv_layers.0.1.running_var : Float(16),
      %conv_layers.0.1.num_batches_tracked : Long(),
      %dense_layers.0.1.weight : Float(512, 1024),
      %dense_layers.0.1.bias : Float(512),
      %dense_layers.1.1.weight : Float(10, 512),
      %dense_layers.1.1.bias : Float(10)):
  %12 : Float(1, 16, 32, 32) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input_0, %conv_layers.0.0.weight, %conv_layers.0.0.bias) # /home/sungmin/Documents/dlTestGen/dlenv/lib/python3.6/site-packages/torch/nn/modules/conv.py:350:0
  %13 : Float(1, 16, 32, 32) = onnx::BatchNormalization[epsilon=1.0000000000000001e-05, momentum=0.90000000000000002](%12, %conv_layers.0.1.weight, %conv_layers.0.1.

DEBUG:onnx2keras:Output TF Layer -> Tensor("12/BiasAdd:0", shape=(?, 16, 32, 32), dtype=float32)
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: BatchNormalization
DEBUG:onnx2keras:node_name: 13
DEBUG:onnx2keras:node_params: {'epsilon': 9.999999747378752e-06, 'momentum': 0.8999999761581421, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 12).
DEBUG:onnx2keras:Check input 1 (name conv_layers.0.1.weight).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name conv_layers.0.1.bias).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 3 (name conv_layers.0.1.running_mean).
DEBUG:onnx2keras:The input not found in layers 

DEBUG:onnx2keras:Check input 2 (name conv_layers.0.0.bias).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer
DEBUG:onnx2keras:Output TF Layer -> Tensor("12_1/BiasAdd:0", shape=(?, 16, 32, 32), dtype=float32)
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: BatchNormalization
DEBUG:onnx2keras:node_name: 13
DEBUG:onnx2keras:node_params: {'epsilon': 9.999999747378752e-06, 'momentum': 0.8999999761581421, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 12).
DEBUG:onnx2keras:Check input 1 (name conv_layers.0.1.weight).
DEBUG:onnx2keras:The input not found in layers / model inputs

graph(%input_0 : Float(1, 3, 32, 32),
      %conv_layers.0.0.weight : Float(16, 3, 3, 3),
      %conv_layers.0.0.bias : Float(16),
      %conv_layers.0.1.weight : Float(16),
      %conv_layers.0.1.bias : Float(16),
      %conv_layers.0.1.running_mean : Float(16),
      %conv_layers.0.1.running_var : Float(16),
      %conv_layers.0.1.num_batches_tracked : Long(),
      %dense_layers.0.1.weight : Float(512, 1024),
      %dense_layers.0.1.bias : Float(512),
      %dense_layers.1.1.weight : Float(10, 512),
      %dense_layers.1.1.bias : Float(10)):
  %12 : Float(1, 16, 32, 32) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input_0, %conv_layers.0.0.weight, %conv_layers.0.0.bias) # /home/sungmin/Documents/dlTestGen/dlenv/lib/python3.6/site-packages/torch/nn/modules/conv.py:350:0
  %13 : Float(1, 16, 32, 32) = onnx::BatchNormalization[epsilon=1.0000000000000001e-05, momentum=0.90000000000000002](%12, %conv_layers.0.1.weight, %conv_layers.0.1.

...

graph(%input_0 : Float(1, 3, 32, 32),
      %conv_layers.0.0.weight : Float(16, 3, 3, 3),
      %conv_layers.0.0.bias : Float(16),
      %conv_layers.0.1.weight : Float(16),
      %conv_layers.0.1.bias : Float(16),
      %conv_layers.0.1.running_mean : Float(16),
      %conv_layers.0.1.running_var : Float(16),
      %conv_layers.0.1.num_batches_tracked : Long(),
      %dense_layers.0.1.weight : Float(512, 1024),
      %dense_layers.0.1.bias : Float(512),
      %dense_layers.1.1.weight : Float(10, 512),
      %dense_layers.1.1.bias : Float(10)):
  %12 : Float(1, 16, 32, 32) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input_0, %conv_layers.0.0.weight, %conv_layers.0.0.bias) # /home/sungmin/Documents/dlTestGen/dlenv/lib/python3.6/site-packages/torch/nn/modules/conv.py:350:0
  %13 : Float(1, 16, 32, 32) = onnx::BatchNormalization[epsilon=1.0000000000000001e-05, momentum=0.90000000000000002](%12, %conv_layers.0.1.weight, %conv_layers.0.1.

DEBUG:onnx2keras:Output TF Layer -> Tensor("12_28/BiasAdd:0", shape=(?, 16, 32, 32), dtype=float32)
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: BatchNormalization
DEBUG:onnx2keras:node_name: 13
DEBUG:onnx2keras:node_params: {'epsilon': 9.999999747378752e-06, 'momentum': 0.8999999761581421, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 12).
DEBUG:onnx2keras:Check input 1 (name conv_layers.0.1.weight).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name conv_layers.0.1.bias).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 3 (name conv_layers.0.1.running_mean).
DEBUG:onnx2keras:The input not found in laye

DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name conv_layers.0.0.bias).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:... found all, continue
DEBUG:onnx2keras:conv:Conv with bias
DEBUG:onnx2keras:conv:2D convolution
DEBUG:onnx2keras:conv:Paddings exist, add ZeroPadding layer


graph(%input_0 : Float(1, 3, 32, 32),
      %conv_layers.0.0.weight : Float(16, 3, 3, 3),
      %conv_layers.0.0.bias : Float(16),
      %conv_layers.0.1.weight : Float(16),
      %conv_layers.0.1.bias : Float(16),
      %conv_layers.0.1.running_mean : Float(16),
      %conv_layers.0.1.running_var : Float(16),
      %conv_layers.0.1.num_batches_tracked : Long(),
      %dense_layers.0.1.weight : Float(512, 1024),
      %dense_layers.0.1.bias : Float(512),
      %dense_layers.1.1.weight : Float(10, 512),
      %dense_layers.1.1.bias : Float(10)):
  %12 : Float(1, 16, 32, 32) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input_0, %conv_layers.0.0.weight, %conv_layers.0.0.bias) # /home/sungmin/Documents/dlTestGen/dlenv/lib/python3.6/site-packages/torch/nn/modules/conv.py:350:0
  %13 : Float(1, 16, 32, 32) = onnx::BatchNormalization[epsilon=1.0000000000000001e-05, momentum=0.90000000000000002](%12, %conv_layers.0.1.weight, %conv_layers.0.1.

DEBUG:onnx2keras:Output TF Layer -> Tensor("12_29/BiasAdd:0", shape=(?, 16, 32, 32), dtype=float32)
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: BatchNormalization
DEBUG:onnx2keras:node_name: 13
DEBUG:onnx2keras:node_params: {'epsilon': 9.999999747378752e-06, 'momentum': 0.8999999761581421, 'change_ordering': False, 'name_policy': None}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all inputs are available:
DEBUG:onnx2keras:Check input 0 (name 12).
DEBUG:onnx2keras:Check input 1 (name conv_layers.0.1.weight).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 2 (name conv_layers.0.1.bias).
DEBUG:onnx2keras:The input not found in layers / model inputs.
DEBUG:onnx2keras:Found in weights, add as a numpy constant.
DEBUG:onnx2keras:Check input 3 (name conv_layers.0.1.running_mean).
DEBUG:onnx2keras:The input not found in laye

In [287]:
fm_mdl_path = "data/models/fmnist_simple.h5"
#gtsrb_mdl_path = "data/models/GTSRB/gtsrb.model.0.wh.0.h5"
gtsrb_mdl_path = "data/models/GTSRB/simple/gtsrb.model.t0.wh.0.h5"

In [43]:
_mdl = load_model(fm_mdl_path, compile = False)
_mdl.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         (None, 1, 784)            0         
_________________________________________________________________
5 (Dense)                    (None, 1, 100)            78500     
_________________________________________________________________
6 (Activation)               (None, 1, 100)            0         
_________________________________________________________________
output_0 (Dense)             (None, 1, 10)             1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


TypeError: 'list' object is not callable

In [47]:
_mdl.layers[1].weights

[<tf.Variable '5_73/kernel:0' shape=(784, 100) dtype=float32>,
 <tf.Variable '5_73/bias:0' shape=(100,) dtype=float32>]

In [288]:
### for keras model
from tensorflow.keras.models import load_model

path_to_keras_model = gtsrb_mdl_path #fm_mdl_path
#dirname = "data/models/faulty_models/fm/all_layers"
dirname = "data/models/faulty_models/GTSRB/all_layers/simple"
os.makedirs(dirname, exist_ok = True)

mdl_key = os.path.basename(path_to_keras_model)[:-3]

for seed_val in range(30):
    mdl = load_model(path_to_keras_model, compile = False)
    # plant seed
    np.random.seed(seed_val)
    
    # reset network
    #org_net.load_state_dict(torch.load(idlm_path))
    
    # inject faults (over all layers)
    #org_net.apply(inject_faults)
    
    for layer in mdl.layers:
        inject_faults_for_keras_mdl(layer)
        
    # save network
    #input_var = torch.randn(1, 3, 32, 32)
    #k_model = pytorch2keras.pytorch_to_keras(org_net, input_var, [(3, 32, 32)], verbose=True)
    #k_model.save('./faulty_models/cifar10_allLayers_seed%d.h5' % seed_val)
    destfile = os.path.join(dirname, "{}_seed{}.h5".format(mdl_key, seed_val))
    print ("Save to {}".format(destfile))
    mdl.save(destfile)


Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed0.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed1.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed2.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed3.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed4.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed5.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed6.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed7.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed8.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed9.h5
Save to data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed10.h5
Save to data/models/faulty_models/GTSRB/al

In [41]:
from tensorflow.keras.models import load_model
import re 

targeting_clname_pattns = ['Dense*', 'Conv*'] #if not target_all else None
is_target = lambda clname,targets: (targets is None) or any([bool(re.match(t,clname)) for t in targets])
    
seed = 10
mdl = load_model(fm_mdl_path, compile = False)
#mdl = load_model(gtsrb_mdl_path, compile = False)
mdl2 = load_model("data/models/faulty_models/fm/all_layers/fmnist_simple_seed{}.h5".format(seed), compile = False)
#mdl2 = load_model("data/models/faulty_models/GTSRB/all_layers/gtsrb.model.0.wh.0_seed{}.h5".format(seed), compile = False)

num = len(mdl.layers)
for i in range(num):
    class_name = type(mdl.layers[i]).__name__
    if is_target(class_name, targeting_clname_pattns):
        w, _ = mdl.layers[i].get_weights()
        new_w, _ = mdl2.layers[i].get_weights()
        
        num_diff = len(np.where(new_w.flatten() != w.flatten())[0])
        total_num = len(w.flatten())
        print ("layer", i, num_diff/total_num)
        

layer 1 0.04924744897959184
layer 3 0.056


In [34]:
mdl.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_0 (InputLayer)         (None, 1, 784)            0         
_________________________________________________________________
5 (Dense)                    (None, 1, 100)            78500     
_________________________________________________________________
6 (Activation)               (None, 1, 100)            0         
_________________________________________________________________
output_0 (Dense)             (None, 1, 10)             1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [20]:
np.where(w - new_w != 0)

(array([  0,   0,   0, ..., 783, 783, 783]),
 array([ 8, 20, 52, ..., 66, 68, 92]))

In [49]:
onow_np - new_weights_np

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        , -0.47774778,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        -0.0920061 ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.10139924,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

## save gts

In [74]:
def get_gts(idx_to_tls, init_mdl, aft_mdl):
    """
    """
    gts = {}
    for idx_to_tl in idx_to_tls: 
        init_weight = init_mdl.layers[idx_to_tl].get_weights()[0]
        aft_weight = aft_mdl.layers[idx_to_tl].get_weights()[0]
        
        ret = np.where(init_weight != aft_weight)
        ret_indices = list(zip(*ret))
        gts[idx_to_tl] = ret_indices
    
    return gts

In [54]:
from run_localise import get_target_weights

In [55]:
c10_target_layes = get_target_weights(None, c10_mdl_path, indices_to_target = None, target_all = True)

In [57]:
fm_target_layes = get_target_weights(None, fm_mdl_path, indices_to_target = None, target_all = True)

In [292]:
gtsrb_target_layes = get_target_weights(None, gtsrb_mdl_path, indices_to_target = None, target_all = True)

In [293]:
fm_mdl_path_fault_file_fm = "data/models/faulty_models/fm/all_layers/fmnist_simple_seed{}.h5"
c10_mdl_path_fault_file_fm = "data/models/faulty_models/c10/cifar10/cifar10_simple_seed{}.h5"
#gtsrb_mdl_path_fault_file_fm = "data/models/faulty_models/GTSRB/all_layers/gtsrb.model.0.wh.0_seed{}.h5"
gtsrb_mdl_path_fault_file_fm = "data/models/faulty_models/GTSRB/all_layers/simple/gtsrb.model.t0.wh.0_seed{}.h5"

In [51]:
c10_mdl_path = "data/models/cifar_simple_90p.h5"

In [294]:
fm_indices_to_tls = list(fm_target_layes.keys())
c10_indices_to_tls = list(c10_target_layes.keys())
gtsrb_indices_to_tls = list(gtsrb_target_layes.keys())

In [296]:
target_fm_mdl = load_model(fm_mdl_path, compile = False)
target_c10_mdl = load_model(c10_mdl_path, compile = False)
target_gtsrb_mdl = load_model(gtsrb_mdl_path, compile = False)

In [297]:
#which = 'fashion_mnist'
#which = 'cifar10'
which = 'GTSRB'

In [76]:
import pandas as pd
import tqdm
import pickle

In [298]:
if which == 'fashion_mnist':
    target_mdl = target_fm_mdl
    target_tl_indices = fm_indices_to_tls
elif which == 'cifar10':
    target_mdl = target_c10_mdl
    target_tl_indices = c10_indices_to_tls
else:
    target_mdl = target_gtsrb_mdl
    target_tl_indices = gtsrb_indices_to_tls
    
gts_dir = os.path.join("data/gts/rq1/simple/{}".format(which))
os.makedirs(gts_dir, exist_ok = True)

for seed in tqdm.tqdm(range(30)):
    if which == 'fashion_mnist':
        mdl = load_model(fm_mdl_path_fault_file_fm.format(seed), compile = False)
    elif which == 'cifar10':
        mdl = load_model(c10_mdl_path_fault_file_fm.format(seed), compile = False)
    else:
        mdl = load_model(gtsrb_mdl_path_fault_file_fm.format(seed), compile = False)
        
    gts = get_gts(target_tl_indices, target_mdl, mdl)
    gts_file = os.path.join(gts_dir, "rq1.seed{}.gts.pkl".format(seed))
    
    #df = pd.DataFrame(gts)
    #df.to_pickle(gts_file)
    with open(gts_file, 'wb') as f:
        pickle.dump(gts, f)

100%|██████████| 30/30 [08:57<00:00, 17.91s/it]


In [ ]:
def compute_roc_auc(ranks, gts):
    from sklearn.metrics import auc,roc_curve 
    from sklearn.preprocessing import MinMaxScaler
    
    scaler = MinMaxScaler()	

    uniq_ranks = sorted(list(set(ranks.reshape(-1,))))

    ranks_of_gts = ranks[gts]
    num_gts = len(ranks_of_gts)

    recall_per_ranks = []
    for i,uniq_rank in enumerate(uniq_ranks):
        cnt_loc = np.sum(ranks_of_gts <= uniq_rank)
        recall_per_ranks.append(cnt_loc/num_gts)

    xs = [int(r) for r in uniq_ranks]
    auc_score = auc(xs, recall_per_ranks)
    
    return auc_score, list(zip(xs, recall_per_ranks))

In [84]:
loc_file = "results/rq1/all_layers/simple_fm/loc.all_cost.loc.0.1.pkl"

In [92]:
fm_target_layes[1][0]

array([[ 0.38260412,  0.34203607,  0.15109418, ...,  0.07083493,
        -0.090018  ,  0.37902394],
       [ 0.08018471,  0.1468847 , -0.3148051 , ...,  0.26775002,
         0.13219263, -0.44124016],
       [ 0.99782073,  0.3959316 , -0.39116275, ...,  0.22424038,
         0.43500382, -0.5753424 ],
       ...,
       [ 0.13608551,  0.04205639,  0.02728352, ...,  0.05676162,
         0.02562602, -0.6894089 ],
       [-0.33116543,  0.21478154,  0.471417  , ...,  0.10401588,
         0.67420214,  0.04792061],
       [ 0.20624009,  0.24127744,  0.44048694, ...,  0.6069383 ,
         0.38830775, -0.80433786]], dtype=float32)

In [93]:
with open(loc_file, 'rb') as f:
    locs = pickle.load(f)
new_locs = []
for loc in locs:
    indices, cost = loc
    idx_to_l, local_idx = indices
    local_idx = np.unravel_index(local_idx, fm_target_layes[idx_to_l][0].shape)
    new_locs.append([[idx_to_l, local_idx], cost])

In [103]:
new_locs

[[[1, (0, 0)], array([3.13106738e-03, 2.67206152e-09])],
 [[1, (0, 1)], array([3.24256904e-02, 7.22222895e-09])],
 [[1, (0, 2)], array([0.00000000e+00, 9.73189086e-10])],
 [[1, (0, 3)], array([4.75183800e-02, 6.29985586e-10])],
 [[1, (0, 4)], array([0.00000000e+00, 4.93253876e-10])],
 [[1, (0, 5)], array([0.00000000e+00, 4.24946496e-09])],
 [[1, (0, 6)], array([5.80421556e-03, 2.51327012e-09])],
 [[1, (0, 7)], array([8.38170946e-03, 5.17119446e-09])],
 [[1, (0, 8)], array([0.00000000e+00, 2.82623413e-09])],
 [[1, (0, 9)], array([6.73324168e-02, 5.27330541e-09])],
 [[1, (0, 10)], array([0.00000000e+00, 6.37885461e-10])],
 [[1, (0, 11)], array([0.00000000e+00, 3.20596464e-10])],
 [[1, (0, 12)], array([4.72060814e-02, 6.13385501e-09])],
 [[1, (0, 13)], array([7.72617874e-04, 2.18397533e-09])],
 [[1, (0, 14)], array([0.0000000e+00, 2.1307218e-10])],
 [[1, (0, 15)], array([0.0000000e+00, 6.5554311e-11])],
 [[1, (0, 16)], array([6.58652335e-02, 1.85680249e-09])],
 [[1, (0, 17)], array([2.488

In [98]:
with open("results/rq1/all_layers/simple_fm/grad/loc.all_cost.loc.0.1.grad.pkl", 'rb') as f:
    grad_locs = pickle.load(f)

In [102]:
grad_locs

[([3, (93, 6)], 14614.842),
 ([3, (93, 4)], 9396.044),
 ([3, (93, 8)], 6033.9316),
 ([3, (93, 0)], 5806.341),
 ([3, (9, 6)], 5498.6074),
 ([3, (93, 2)], 4745.653),
 ([3, (45, 6)], 4572.8706),
 ([3, (50, 6)], 4295.22),
 ([3, (71, 6)], 4265.6543),
 ([3, (45, 4)], 3572.9094),
 ([3, (9, 4)], 3434.061),
 ([3, (9, 8)], 3323.628),
 ([3, (5, 4)], 2947.9844),
 ([3, (5, 6)], 2673.6934),
 ([3, (45, 8)], 2503.706),
 ([3, (46, 7)], 2474.5686),
 ([3, (27, 7)], 2474.4722),
 ([3, (37, 7)], 2395.8098),
 ([3, (33, 7)], 2303.5208),
 ([3, (86, 4)], 2232.5571),
 ([3, (67, 6)], 2217.7637),
 ([3, (74, 4)], 2179.2095),
 ([3, (42, 7)], 2129.6577),
 ([3, (71, 4)], 2101.9414),
 ([3, (74, 6)], 2086.6382),
 ([3, (86, 6)], 2050.6292),
 ([3, (50, 4)], 2048.1226),
 ([3, (67, 4)], 2032.7578),
 ([3, (9, 2)], 1962.869),
 ([3, (50, 8)], 1961.8867),
 ([3, (45, 2)], 1959.9155),
 ([3, (1, 6)], 1951.5448),
 ([3, (71, 8)], 1945.7554),
 ([1, (343, 48)], 1933.6929),
 ([1, (371, 48)], 1929.2078),
 ([1, (315, 48)], 1922.9739),
 (

In [116]:
with open("results/rq1/all_layers/simple_fm/random/loc.all_cost.loc.0.1.random.pkl", 'rb') as f:
    random_locs = pickle.load(f)

In [127]:
random_locs

[(1, (0, 0)),
 (1, (0, 1)),
 (1, (0, 2)),
 (1, (0, 3)),
 (1, (0, 4)),
 (1, (0, 5)),
 (1, (0, 6)),
 (1, (0, 7)),
 (1, (0, 8)),
 (1, (0, 9)),
 (1, (0, 10)),
 (1, (0, 11)),
 (1, (0, 12)),
 (1, (0, 13)),
 (1, (0, 14)),
 (1, (0, 15)),
 (1, (0, 16)),
 (1, (0, 17)),
 (1, (0, 18)),
 (1, (0, 19)),
 (1, (0, 20)),
 (1, (0, 21)),
 (1, (0, 22)),
 (1, (0, 23)),
 (1, (0, 24)),
 (1, (0, 25)),
 (1, (0, 26)),
 (1, (0, 27)),
 (1, (0, 28)),
 (1, (0, 29)),
 (1, (0, 30)),
 (1, (0, 31)),
 (1, (0, 32)),
 (1, (0, 33)),
 (1, (0, 34)),
 (1, (0, 35)),
 (1, (0, 36)),
 (1, (0, 37)),
 (1, (0, 38)),
 (1, (0, 39)),
 (1, (0, 40)),
 (1, (0, 41)),
 (1, (0, 42)),
 (1, (0, 43)),
 (1, (0, 44)),
 (1, (0, 45)),
 (1, (0, 46)),
 (1, (0, 47)),
 (1, (0, 48)),
 (1, (0, 49)),
 (1, (0, 50)),
 (1, (0, 51)),
 (1, (0, 52)),
 (1, (0, 53)),
 (1, (0, 54)),
 (1, (0, 55)),
 (1, (0, 56)),
 (1, (0, 57)),
 (1, (0, 58)),
 (1, (0, 59)),
 (1, (0, 60)),
 (1, (0, 61)),
 (1, (0, 62)),
 (1, (0, 63)),
 (1, (0, 64)),
 (1, (0, 65)),
 (1, (0, 66)),
 (1, 

In [272]:
def get_weight_and_cost(loc_which, seed, loc_file, target_weights, method = 'max'):
    """
    """
    from scipy.stats import rankdata
    
    with open(loc_file, 'rb') as f:
        locs = pickle.load(f)
    
    if loc_which == 'localiser':
        new_locs = []
        for loc in locs:
            indices, cost = loc
            idx_to_l, local_idx = indices
            local_idx = np.unravel_index(local_idx, target_weights[idx_to_l][0].shape)
            new_locs.append([[idx_to_l, local_idx], cost])
        
        costs = [vs[1] for vs in new_locs]
        ranks = rankdata(costs, method = method)
        indices = [vs[0] for vs in new_locs]
        
        #print (len(ranks), len(indices))
        ret_lst = compute_pareto(np.asarray(costs), indices)
        
        pairs = {}
        for i,r in enumerate(ret_lst):
            #rint (indices[i])
            pairs[r] = i+1 # 1 ~ num
            
    elif loc_which == 'gradient_loss':
        costs = [-vs[-1] for vs in locs]
        ranks = rankdata(costs, method = method)
        indices = [vs[0] for vs in locs]
        
        pairs = {}
        for i,r in enumerate(ranks):
            pairs[tuple(indices[i])] = r
    else:
        np.random.seed(seed)
        nindices = np.arange(len(locs))
        np.random.shuffle(nindices)
        
        pairs = {}
        for i,aloc in enumerate(nindices):
            pairs[tuple(locs[aloc])] = i+1 # 1 ~ num
    
    return pairs

In [242]:
def compute_pareto(costs, curr_nodes_to_lookat):
    ret_lst = []
    ret_front_lst = []
     
    while len(curr_nodes_to_lookat) > 0:
        _costs = costs.copy()
        is_efficient = np.arange(costs.shape[0])
        next_point_index = 0 # Next index in the is_efficient array to search for

        while next_point_index < len(_costs):
            nondominated_point_mask = np.any(_costs > _costs[next_point_index], axis=1)
            nondominated_point_mask[next_point_index] = True
            is_efficient = is_efficient[nondominated_point_mask]  # Remove dominated points
            _costs = _costs[nondominated_point_mask]
            next_point_index = np.sum(nondominated_point_mask[:next_point_index])+1
            
        current_ret = [tuple(v) for v in np.asarray(curr_nodes_to_lookat)[is_efficient]]
        ret_lst.extend(current_ret)
            
        ret_front_lst.append(current_ret)
        # remove selected items (non-dominated ones)
        curr_nodes_to_lookat = np.delete(curr_nodes_to_lookat, is_efficient, 0)
        costs = np.delete(costs, is_efficient, 0)
    
    return ret_lst
        

In [230]:
df = pd.DataFrame([[tuple(vs[0]), vs[1][0], vs[1][1]] for vs in new_locs])
df

,0,1,2
0,"(1, (0, 0))",0.003131,2.672062e-09
1,"(1, (0, 1))",0.032426,7.222229e-09
2,"(1, (0, 2))",0.000000,9.731891e-10
3,"(1, (0, 3))",0.047518,6.299856e-10
4,"(1, (0, 4))",0.000000,4.932539e-10
5,"(1, (0, 5))",0.000000,4.249465e-09
6,"(1, (0, 6))",0.005804,2.513270e-09
7,"(1, (0, 7))",0.008382,5.171194e-09
8,"(1, (0, 8))",0.000000,2.826234e-09
9,"(1, (0, 9))",0.067332,5.273305e-09


In [243]:
ret_lst = compute_pareto(np.asarray([vs[1] for vs in new_locs]), [vs[0] for vs in new_locs])

In [254]:
ret_lst

[(3, (9, 8)),
 (3, (93, 6)),
 (3, (93, 7)),
 (3, (93, 9)),
 (3, (5, 3)),
 (3, (16, 6)),
 (3, (48, 2)),
 (3, (93, 0)),
 (3, (93, 2)),
 (3, (93, 4)),
 (3, (93, 5)),
 (3, (1, 8)),
 (3, (5, 4)),
 (3, (9, 2)),
 (3, (9, 6)),
 (3, (21, 6)),
 (3, (45, 2)),
 (3, (45, 6)),
 (3, (45, 9)),
 (3, (62, 6)),
 (3, (66, 0)),
 (3, (66, 8)),
 (3, (67, 8)),
 (3, (74, 8)),
 (3, (93, 3)),
 (3, (93, 8)),
 (3, (5, 7)),
 (3, (9, 7)),
 (3, (41, 6)),
 (3, (45, 3)),
 (3, (45, 4)),
 (3, (50, 5)),
 (3, (50, 6)),
 (3, (59, 4)),
 (3, (62, 7)),
 (3, (71, 1)),
 (3, (71, 6)),
 (3, (74, 5)),
 (3, (76, 6)),
 (3, (86, 0)),
 (3, (1, 7)),
 (3, (5, 2)),
 (3, (5, 6)),
 (3, (9, 4)),
 (3, (9, 9)),
 (3, (16, 2)),
 (3, (50, 9)),
 (3, (55, 5)),
 (3, (74, 2)),
 (3, (16, 0)),
 (3, (37, 0)),
 (3, (45, 1)),
 (3, (45, 8)),
 (3, (46, 5)),
 (3, (48, 6)),
 (3, (55, 8)),
 (3, (59, 6)),
 (3, (66, 1)),
 (3, (71, 0)),
 (3, (71, 4)),
 (3, (86, 4)),
 (3, (1, 4)),
 (3, (16, 9)),
 (3, (27, 7)),
 (3, (37, 1)),
 (3, (40, 5)),
 (3, (46, 7)),
 (3, (50,

In [217]:
df.loc[df[0] == (3,(9,8))]

,0,1
78498,"(3, (9, 8))","(3323.6279296875, 0.01753034889698214)"


In [222]:
df.rank(axis = 1, method = 'max').iloc[78498].values

array([1., 2.])

In [232]:
df.sort_values(by=[2,1], ascending = False)

,0,1,2
79339,"(3, (93, 9))",17.565239,2.878784e-02
79337,"(3, (93, 7))",46.013561,2.651494e-02
78498,"(3, (9, 8))",3323.627930,1.753035e-02
79335,"(3, (93, 5))",27.272110,1.649347e-02
78453,"(3, (5, 3))",153.317612,1.080111e-02
78859,"(3, (45, 9))",40.848118,8.727506e-03
79111,"(3, (71, 1))",6.814991,8.679789e-03
78499,"(3, (9, 9))",5.095457,8.539372e-03
78497,"(3, (9, 7))",12.148599,8.417979e-03
78882,"(3, (48, 2))",396.964294,8.025576e-03


In [200]:
list(pairs.items())

[((3, (93, 6)), 1),
 ((3, (93, 4)), 2),
 ((3, (93, 8)), 3),
 ((3, (93, 0)), 4),
 ((3, (9, 6)), 5),
 ((3, (93, 2)), 6),
 ((3, (45, 6)), 7),
 ((3, (50, 6)), 8),
 ((3, (71, 6)), 9),
 ((3, (45, 4)), 10),
 ((3, (9, 4)), 11),
 ((3, (9, 8)), 12),
 ((3, (5, 4)), 13),
 ((3, (5, 6)), 14),
 ((3, (45, 8)), 15),
 ((3, (46, 7)), 16),
 ((3, (27, 7)), 17),
 ((3, (37, 7)), 18),
 ((3, (33, 7)), 19),
 ((3, (86, 4)), 20),
 ((3, (67, 6)), 21),
 ((3, (74, 4)), 22),
 ((3, (42, 7)), 23),
 ((3, (71, 4)), 24),
 ((3, (74, 6)), 25),
 ((3, (86, 6)), 26),
 ((3, (50, 4)), 27),
 ((3, (67, 4)), 28),
 ((3, (9, 2)), 29),
 ((3, (50, 8)), 30),
 ((3, (45, 2)), 31),
 ((3, (1, 6)), 32),
 ((3, (71, 8)), 33),
 ((1, (343, 48)), 34),
 ((1, (371, 48)), 35),
 ((1, (315, 48)), 36),
 ((3, (34, 7)), 37),
 ((3, (55, 6)), 38),
 ((1, (399, 48)), 39),
 ((1, (287, 48)), 40),
 ((3, (50, 0)), 41),
 ((1, (288, 48)), 42),
 ((1, (441, 48)), 43),
 ((1, (427, 48)), 44),
 ((1, (469, 48)), 45),
 ((1, (413, 48)), 46),
 ((1, (316, 48)), 47),
 ((1, (

In [183]:
df.to_csv("a.csv", sep = ";", header = False, index = False)
adf = pd.read_csv("a.csv", sep =";", header = None)
adf

,0,1
0,"(3, (93, 6))",1
1,"(3, (93, 4))",2
2,"(3, (93, 8))",3
3,"(3, (93, 0))",4
4,"(3, (9, 6))",5
5,"(3, (93, 2))",6
6,"(3, (45, 6))",7
7,"(3, (50, 6))",8
8,"(3, (71, 6))",9
9,"(3, (45, 4))",10


In [174]:
df.to_pickle("a.pkl")

In [258]:
seed = 0
loc_file = "results/rq1/all_layers/simple_fm/grad/loc.all_cost.loc.0.1.grad.pkl"
loc_which = 'gradient_loss'
target_weights = fm_target_layes

pairs = get_weight_and_cost(loc_which, seed, loc_file, target_weights, method = 'max')

In [259]:
pairs

{(3, (93, 6)): 1,
 (3, (93, 4)): 2,
 (3, (93, 8)): 3,
 (3, (93, 0)): 4,
 (3, (9, 6)): 5,
 (3, (93, 2)): 6,
 (3, (45, 6)): 7,
 (3, (50, 6)): 8,
 (3, (71, 6)): 9,
 (3, (45, 4)): 10,
 (3, (9, 4)): 11,
 (3, (9, 8)): 12,
 (3, (5, 4)): 13,
 (3, (5, 6)): 14,
 (3, (45, 8)): 15,
 (3, (46, 7)): 16,
 (3, (27, 7)): 17,
 (3, (37, 7)): 18,
 (3, (33, 7)): 19,
 (3, (86, 4)): 20,
 (3, (67, 6)): 21,
 (3, (74, 4)): 22,
 (3, (42, 7)): 23,
 (3, (71, 4)): 24,
 (3, (74, 6)): 25,
 (3, (86, 6)): 26,
 (3, (50, 4)): 27,
 (3, (67, 4)): 28,
 (3, (9, 2)): 29,
 (3, (50, 8)): 30,
 (3, (45, 2)): 31,
 (3, (1, 6)): 32,
 (3, (71, 8)): 33,
 (1, (343, 48)): 34,
 (1, (371, 48)): 35,
 (1, (315, 48)): 36,
 (3, (34, 7)): 37,
 (3, (55, 6)): 38,
 (1, (399, 48)): 39,
 (1, (287, 48)): 40,
 (3, (50, 0)): 41,
 (1, (288, 48)): 42,
 (1, (441, 48)): 43,
 (1, (427, 48)): 44,
 (1, (469, 48)): 45,
 (1, (413, 48)): 46,
 (1, (316, 48)): 47,
 (1, (426, 48)): 48,
 (1, (385, 48)): 49,
 (1, (259, 48)): 50,
 (1, (454, 48)): 51,
 (1, (497, 48)): 

In [256]:
seed = 0
loc_file = "results/rq1/all_layers/simple_fm/loc.all_cost.loc.0.1.pkl"
loc_which = 'localiser'
target_weights = fm_target_layes

pairs_loc = get_weight_and_cost(loc_which, seed, loc_file, target_weights, method = 'max')

In [260]:
pairs_loc

{(3, (9, 8)): 1,
 (3, (93, 6)): 2,
 (3, (93, 7)): 3,
 (3, (93, 9)): 4,
 (3, (5, 3)): 5,
 (3, (16, 6)): 6,
 (3, (48, 2)): 7,
 (3, (93, 0)): 8,
 (3, (93, 2)): 9,
 (3, (93, 4)): 10,
 (3, (93, 5)): 11,
 (3, (1, 8)): 12,
 (3, (5, 4)): 13,
 (3, (9, 2)): 14,
 (3, (9, 6)): 15,
 (3, (21, 6)): 16,
 (3, (45, 2)): 17,
 (3, (45, 6)): 18,
 (3, (45, 9)): 19,
 (3, (62, 6)): 20,
 (3, (66, 0)): 21,
 (3, (66, 8)): 22,
 (3, (67, 8)): 23,
 (3, (74, 8)): 24,
 (3, (93, 3)): 25,
 (3, (93, 8)): 26,
 (3, (5, 7)): 27,
 (3, (9, 7)): 28,
 (3, (41, 6)): 29,
 (3, (45, 3)): 30,
 (3, (45, 4)): 31,
 (3, (50, 5)): 32,
 (3, (50, 6)): 33,
 (3, (59, 4)): 34,
 (3, (62, 7)): 35,
 (3, (71, 1)): 36,
 (3, (71, 6)): 37,
 (3, (74, 5)): 38,
 (3, (76, 6)): 39,
 (3, (86, 0)): 40,
 (3, (1, 7)): 41,
 (3, (5, 2)): 42,
 (3, (5, 6)): 43,
 (3, (9, 4)): 44,
 (3, (9, 9)): 45,
 (3, (16, 2)): 46,
 (3, (50, 9)): 47,
 (3, (55, 5)): 48,
 (3, (74, 2)): 49,
 (3, (16, 0)): 50,
 (3, (37, 0)): 51,
 (3, (45, 1)): 52,
 (3, (45, 8)): 53,
 (3, (46, 5)): 

In [279]:
seed = 10
loc_file = "results/rq1/all_layers/simple_fm/random/loc.all_cost.loc.0.1.random.pkl"
loc_which = 'random'
target_weights = fm_target_layes

pairs_rd = get_weight_and_cost(loc_which, seed, loc_file, target_weights, method = 'max')

In [280]:
pairs_rd

{(1, (393, 24)): 1,
 (1, (542, 78)): 2,
 (1, (74, 83)): 3,
 (1, (614, 36)): 4,
 (1, (596, 47)): 5,
 (1, (375, 46)): 6,
 (1, (628, 72)): 7,
 (1, (586, 58)): 8,
 (1, (580, 92)): 9,
 (1, (311, 14)): 10,
 (1, (348, 85)): 11,
 (1, (395, 62)): 12,
 (1, (150, 11)): 13,
 (1, (578, 30)): 14,
 (1, (237, 3)): 15,
 (1, (173, 27)): 16,
 (1, (699, 8)): 17,
 (1, (542, 5)): 18,
 (1, (431, 54)): 19,
 (1, (306, 10)): 20,
 (1, (287, 37)): 21,
 (1, (264, 7)): 22,
 (1, (430, 58)): 23,
 (1, (506, 78)): 24,
 (1, (7, 66)): 25,
 (1, (240, 32)): 26,
 (1, (588, 61)): 27,
 (1, (405, 47)): 28,
 (1, (740, 91)): 29,
 (1, (666, 86)): 30,
 (1, (92, 11)): 31,
 (1, (536, 61)): 32,
 (1, (72, 13)): 33,
 (1, (65, 69)): 34,
 (1, (496, 25)): 35,
 (1, (176, 3)): 36,
 (1, (551, 35)): 37,
 (1, (624, 52)): 38,
 (1, (244, 10)): 39,
 (1, (167, 37)): 40,
 (1, (508, 56)): 41,
 (1, (439, 55)): 42,
 (1, (742, 9)): 43,
 (1, (488, 66)): 44,
 (1, (708, 31)): 45,
 (1, (210, 4)): 46,
 (1, (51, 60)): 47,
 (1, (610, 28)): 48,
 (1, (174, 21))

In [186]:
loc_dir = "results/rq1/all_layers/simple_fm"

loc_loc_file = "loc.all_cost.loc.{}.1.pkl"
loc_grad_file = "grad/loc.all_cost.loc.{}.1.grad.pkl"
loc_random_file = "random/loc.all_cost.loc.{}.1.random.pkl"

In [285]:
which = 'fashion_mnist'
#loc_which = 'localiser' 
loc_which = 'gradient_loss'
#loc_which = 'random'

In [286]:
if which == 'fashion_mnist':
    target_weights = fm_target_layes
    loc_dir = "results/rq1/all_layers/simple_fm"
elif which == 'cifar10':
    target_weights = c10_target_layes
    loc_dir = "results/rq1/all_layers/simple_cm"
#else:
#    target_weights = gtsrb_target_layes
#    loc_dir = "results/rq1/all_layers/simple_fm"

if loc_which == 'localiser':
    loc_file = loc_loc_file
elif loc_which == 'gradient_loss':
    loc_file = loc_grad_file
else:
    loc_file = loc_random_file
    
dest = os.path.join(loc_dir, "pairs/{}".format(loc_which))
os.makedirs(dest, exist_ok = True)

for seed in tqdm.tqdm(range(30)):
    curr_loc_file = os.path.join(loc_dir, loc_file.format(seed))
    pairs = get_weight_and_cost(loc_which, seed, curr_loc_file, target_weights, method = 'max')
    df = pd.DataFrame(list(pairs.items()))
    pairfile = os.path.join(dest, "{}.pairs.csv".format(seed))
    
    df.to_csv(pairfile, sep = ";", header = False, index = False)

100%|██████████| 30/30 [00:18<00:00,  1.66it/s]


In [135]:
grad_locs

[([3, (93, 6)], 14614.842),
 ([3, (93, 4)], 9396.044),
 ([3, (93, 8)], 6033.9316),
 ([3, (93, 0)], 5806.341),
 ([3, (9, 6)], 5498.6074),
 ([3, (93, 2)], 4745.653),
 ([3, (45, 6)], 4572.8706),
 ([3, (50, 6)], 4295.22),
 ([3, (71, 6)], 4265.6543),
 ([3, (45, 4)], 3572.9094),
 ([3, (9, 4)], 3434.061),
 ([3, (9, 8)], 3323.628),
 ([3, (5, 4)], 2947.9844),
 ([3, (5, 6)], 2673.6934),
 ([3, (45, 8)], 2503.706),
 ([3, (46, 7)], 2474.5686),
 ([3, (27, 7)], 2474.4722),
 ([3, (37, 7)], 2395.8098),
 ([3, (33, 7)], 2303.5208),
 ([3, (86, 4)], 2232.5571),
 ([3, (67, 6)], 2217.7637),
 ([3, (74, 4)], 2179.2095),
 ([3, (42, 7)], 2129.6577),
 ([3, (71, 4)], 2101.9414),
 ([3, (74, 6)], 2086.6382),
 ([3, (86, 6)], 2050.6292),
 ([3, (50, 4)], 2048.1226),
 ([3, (67, 4)], 2032.7578),
 ([3, (9, 2)], 1962.869),
 ([3, (50, 8)], 1961.8867),
 ([3, (45, 2)], 1959.9155),
 ([3, (1, 6)], 1951.5448),
 ([3, (71, 8)], 1945.7554),
 ([1, (343, 48)], 1933.6929),
 ([1, (371, 48)], 1929.2078),
 ([1, (315, 48)], 1922.9739),
 (

In [134]:
pairs

{(3, (93, 6)): 1,
 (3, (93, 4)): 2,
 (3, (93, 8)): 3,
 (3, (93, 0)): 4,
 (3, (9, 6)): 5,
 (3, (93, 2)): 6,
 (3, (45, 6)): 7,
 (3, (50, 6)): 8,
 (3, (71, 6)): 9,
 (3, (45, 4)): 10,
 (3, (9, 4)): 11,
 (3, (9, 8)): 12,
 (3, (5, 4)): 13,
 (3, (5, 6)): 14,
 (3, (45, 8)): 15,
 (3, (46, 7)): 16,
 (3, (27, 7)): 17,
 (3, (37, 7)): 18,
 (3, (33, 7)): 19,
 (3, (86, 4)): 20,
 (3, (67, 6)): 21,
 (3, (74, 4)): 22,
 (3, (42, 7)): 23,
 (3, (71, 4)): 24,
 (3, (74, 6)): 25,
 (3, (86, 6)): 26,
 (3, (50, 4)): 27,
 (3, (67, 4)): 28,
 (3, (9, 2)): 29,
 (3, (50, 8)): 30,
 (3, (45, 2)): 31,
 (3, (1, 6)): 32,
 (3, (71, 8)): 33,
 (1, (343, 48)): 34,
 (1, (371, 48)): 35,
 (1, (315, 48)): 36,
 (3, (34, 7)): 37,
 (3, (55, 6)): 38,
 (1, (399, 48)): 39,
 (1, (287, 48)): 40,
 (3, (50, 0)): 41,
 (1, (288, 48)): 42,
 (1, (441, 48)): 43,
 (1, (427, 48)): 44,
 (1, (469, 48)): 45,
 (1, (413, 48)): 46,
 (1, (316, 48)): 47,
 (1, (426, 48)): 48,
 (1, (385, 48)): 49,
 (1, (259, 48)): 50,
 (1, (454, 48)): 51,
 (1, (497, 48)): 